# Check-worthiness detection using Large Language Models

First, the necessary python modules are imported

In [1]:
%load_ext autoreload

from claimbuster_utils import load_claimbuster_dataset
from checkthat_utils import load_check_that_dataset
from tqdm.auto import tqdm
import json
import numpy as np
import re
import torch
import pandas as pd
from llm import load_huggingface_model, HuggingFaceModel, run_llm_cross_validation, ThresholdOptimizer
import os
from error_analysis import generate_error_analysis_report
from dataset_utils import generate_cross_validation_datasets

c:\Users\matsb\anaconda3\envs\fake-news-detection\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load model

In [8]:
%autoreload
model_id = HuggingFaceModel.MIXTRAL_INSTRUCT
pipe = load_huggingface_model(model_id)


config.json: 100%|██████████| 720/720 [00:00<?, ?B/s] 


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

## Zero-shot classification

### ClaimBuster

In [7]:
with open("../prompts/ClaimBuster/standard/zero-shot.txt", "r") as f:
    instruction = f.read().replace("\n", "")
use_contextual = False
data = load_claimbuster_dataset(
    "../data/ClaimBuster_Datasets/datasets",
    use_contextual_features=use_contextual,
    debate_transcripts_folder="../data/ClaimBuster_Datasets/debate_transcripts",
)[:10]


texts = data["Text"]
if use_contextual is False:
    prompts = [f"{instruction} '''{text}'''" for text in texts]
    zeroshot_output = f"../results/ClaimBuster/{model_id.name}/zeroshot/zeroshot1.csv"
    os.makedirs(os.path.dirname(zeroshot_output), exist_ok=True)
else:
    contexts = data["previous_sentences"].tolist()
    prompts = [
        f"{instruction} For context, the following senteces were said prior to the one in question: {context} Only evaluate the check-worthiness of the following sentence: '''{text}'''"
        for text, context in zip(texts, contexts)
    ]
    zeroshot_output = "../results/ClaimBuster/zeroshot_contextual.csv"


class ProgressDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, idx):
        return self.dataset[idx]

    def __len__(self):
        return len(self.dataset)


prompts_data = ProgressDataset(prompts)
print(pipe.tokenizer.encode(prompts_data))

dataset_with_scores = data.copy()

display(data.head())
dict_matcher = re.compile(r"{.*}")
score_matcher = re.compile(r"([Ss]core[^\d]*)(\d+)")
non_check_worthy_matcher = re.compile(
    r"(non-checkworthy)|(not check-worthy)|(non check-worthy)"
)

responses = pipe(prompts_data, batch_size=128)
for index, result in enumerate(tqdm(responses, total=len(prompts))):
    response = result[0]["generated_text"].replace("\n", "")
    dataset_index = data.index[index]
    try:
        parsed_json = json.loads(dict_matcher.search(response).group(0))
        dataset_with_scores.loc[dataset_index, "score"] = parsed_json["score"]
        dataset_with_scores.loc[dataset_index, "reasoning"] = parsed_json["reasoning"]
    except (json.decoder.JSONDecodeError, AttributeError, KeyError) as e:
        # Try to find score
        score = score_matcher.search(response)
        if score is not None:
            score = score[2]
        else:
            score = 0.0 if non_check_worthy_matcher.search(response) else np.nan
        dataset_with_scores.loc[dataset_index, "score"] = score
        dataset_with_scores.loc[dataset_index, "reasoning"] = response
        continue
# Set the following column order: Verdict, score, Text, reasoning, previous_sentences
columns =  ["Verdict", "score", "Text", "reasoning"]
if use_contextual:
    columns.append("previous_sentences")
dataset_with_scores = dataset_with_scores[columns]
dataset_with_scores.to_csv(zeroshot_output, index=True)

NameError: name 'pipe' is not defined

#### Cross validation

In [28]:
%autoreload

crossval_folder = "../data/ClaimBuster_Datasets/crossval"
for model_id in HuggingFaceModel:
    print(f"Running 4 fold cross validation for model {model_id.name}")
    dataset_path = f"../results/ClaimBuster/{model_id.name}/zeroshot/zeroshot1.csv"
    dataset_with_scores = pd.read_csv(dataset_path, index_col=0)
    save_folder = f"../results/ClaimBuster/{model_id.name}/zeroshot"
    result = run_llm_cross_validation(
        data=dataset_with_scores, crossval_folder=crossval_folder, save_folder=save_folder
    )
    
    display(result)

Running 4 fold cross validation for model MISTRAL_7B_INSTRUCT


FileNotFoundError: [Errno 2] No such file or directory: '../results/ClaimBuster/MISTRAL_7B_INSTRUCT/zeroshot/zeroshot1.csv'

#### Error analysis

In [99]:
%autoreload
mistral_results = pd.read_csv(f"../results/ClaimBuster/{HuggingFaceModel.MISTRAL_7B_INSTRUCT.name}/zeroshot1.csv", index_col=0)
mixtral_results = pd.read_csv(f"../results/ClaimBuster/{HuggingFaceModel.MIXTRAL_INSTRUCT.name}/zeroshot1.csv", index_col=0)
results = [mistral_results, mixtral_results]
models = [HuggingFaceModel.MISTRAL_7B_INSTRUCT, HuggingFaceModel.MIXTRAL_INSTRUCT]
generate_error_analysis_report(
    results=results,
    models=models,
    folder_path=f"../results/ClaimBuster"
)

##################################################
#              MISTRAL_7B_INSTRUCT               #
#              False positives: 413              #
#              False negatives: 624              #
#              Empty predictions: 0              #
#             Wrong output format: 0             #
##################################################
#                MIXTRAL_INSTRUCT                #
#             False positives: 1193              #
#              False negatives: 388              #
#              Empty predictions: 8              #
#            Wrong output format: 259            #
##################################################
#                     Total                      #
#             False positives: 1324              #
#              False negatives: 757              #
#        Overlapping false positives: 282        #
#        Overlapping false negatives: 255        #
##################################################


### CheckThat 2021 Task 1a Tweets

In [3]:
%autoreload
print(torch.cuda.memory_summary())
with open("../prompts/CheckThat/standard/zero-shot.txt", "r") as f:
    instruction = f.read().replace("\n", "")
data = load_check_that_dataset(
    "../data/CheckThat2021Task1a",
)

texts = data["tweet_text"]
prompts = [f"{instruction} '''{text}'''" for text in texts]
zeroshot_output = f"../results/CheckThat/{model_id.name}/zeroshot1.csv"
os.makedirs(os.path.dirname(zeroshot_output), exist_ok=True)

class ProgressDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, idx):
        return self.dataset[idx]

    def __len__(self):
        return len(self.dataset)


prompts_data = ProgressDataset(prompts)

dataset_with_scores = data.copy()

display(data.head())
dict_matcher = re.compile(r"{.*}")
score_matcher = re.compile(r"([Ss]core[^\d]*)(\d+)")
non_check_worthy_matcher = re.compile(
    r"(non-checkworthy)|(not check-worthy)|(non check-worthy)"
)

with torch.no_grad():
    responses = pipe(prompts_data, batch_size=128)
    for index, result in enumerate(tqdm(responses, total=len(prompts))):
        response = result[0]["generated_text"].replace("\n", "")
        dataset_index = data.index[index]
        try:
            parsed_json = json.loads(dict_matcher.search(response).group(0))
            dataset_with_scores.loc[dataset_index, "score"] = parsed_json["score"]
            dataset_with_scores.loc[dataset_index, "reasoning"] = parsed_json["reasoning"]
        except (json.decoder.JSONDecodeError, AttributeError, KeyError) as e:
            # Try to find score
            score = score_matcher.search(response)
            if score is not None:
                score = score[2]
            else:
                score = 0.0 if non_check_worthy_matcher.search(response) else np.nan
            dataset_with_scores.loc[dataset_index, "score"] = score
            dataset_with_scores.loc[dataset_index, "reasoning"] = response
            continue
columns =  ["check_worthiness", "score", "tweet_text", "reasoning"]
dataset_with_scores = dataset_with_scores[columns]
dataset_with_scores.to_csv(zeroshot_output, index=True)

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  25981 MiB |  25981 MiB | 114559 MiB |  88578 MiB |
|       from large pool |  25899 MiB |  25899 MiB | 114475 MiB |  88576 MiB |
|       from small pool |     81 MiB |     81 MiB |     83 MiB |      2 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  25981 MiB |  25981 MiB | 114559 MiB |  88578 MiB |
|       from large pool |  25899 MiB |  25899 MiB | 114475 MiB |

,topic_id,tweet_url,tweet_text,claim,check_worthiness
tweet_id,,,,,
1234964653014384644,covid-19,https://twitter.com/EricTrump/status/123496465...,Since this will never get reported by the medi...,1,1
1234869939720216578,covid-19,https://twitter.com/RealJamesWoods/status/1234...,"Thanks, #MichaelBloomberg. Here’s a handy litt...",0,0
1234873136304267267,covid-19,https://twitter.com/hayxsmith/status/123487313...,"Folks, when you say ""The corona virus isn't a ...",0,0
1235071285027147776,covid-19,https://twitter.com/ipspankajnain/status/12350...,Just 1 case of Corona Virus in India and peop...,1,0
1234911110861594624,covid-19,https://twitter.com/PressSec/status/1234911110...,President @realDonaldTrump made a commitment...,1,1


  0%|          | 1/1172 [02:12<42:59:55, 132.19s/it]/tmp/ipykernel_1771071/3923584380.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataset_with_scores.loc[dataset_index, "score"] = score
100%|██████████| 1172/1172 [22:23<00:00,  1.15s/it] 


#### Cross validation

In [26]:
%autoreload
crossval_folder = "../data/CheckThat2021Task1a/crossval"
for model_id in HuggingFaceModel:
    print(f"Running 4 fold cross validation for model {model_id.name}")
    dataset_path = f"../results/CheckThat/{model_id.name}/zeroshot/zeroshot1.csv"
    dataset_with_scores = pd.read_csv(dataset_path, index_col=0)
    save_folder = f"../results/CheckThat/{model_id.name}/zeroshot"
    result = run_llm_cross_validation(
        data=dataset_with_scores,
        label_column="check_worthiness",
        crossval_folder=crossval_folder,
        save_folder=save_folder,
    )
    display(result)
    


Running 4 fold cross validation for model MISTRAL_7B_INSTRUCT


,accuracy,0_precision,0_recall,0_f1-score,1_precision,1_recall,1_f1-score,macro avg_precision,macro avg_recall,macro avg_f1-score,weighted avg_precision,weighted avg_recall,weighted avg_f1-score
0,0.645051,0.828402,0.651163,0.729167,0.395161,0.628205,0.485149,0.611782,0.639684,0.607158,0.713069,0.645051,0.664206
1,0.607509,0.874074,0.546296,0.672365,0.379747,0.779221,0.510638,0.626910,0.662759,0.591501,0.744166,0.607509,0.629863
2,0.672355,0.840909,0.685185,0.755102,0.418803,0.636364,0.505155,0.629856,0.660774,0.630128,0.729980,0.672355,0.689416
3,0.651877,0.835294,0.657407,0.735751,0.398374,0.636364,0.490000,0.616834,0.646886,0.612876,0.720472,0.651877,0.671168
Average,0.644198,0.844670,0.635013,0.723096,0.398021,0.670038,0.497735,0.621346,0.652526,0.610416,0.726922,0.644198,0.663663


Running 4 fold cross validation for model MIXTRAL_INSTRUCT


,accuracy,0_precision,0_recall,0_f1-score,1_precision,1_recall,1_f1-score,macro avg_precision,macro avg_recall,macro avg_f1-score,weighted avg_precision,weighted avg_recall,weighted avg_f1-score
0,0.726962,0.816901,0.809302,0.813084,0.487500,0.500000,0.493671,0.652201,0.654651,0.653377,0.729211,0.726962,0.728053
1,0.778157,0.864734,0.828704,0.846336,0.569767,0.636364,0.601227,0.717251,0.732534,0.723781,0.787217,0.778157,0.781921
2,0.744027,0.847291,0.796296,0.821002,0.511111,0.597403,0.550898,0.679201,0.696849,0.685950,0.758943,0.744027,0.750019
3,0.754266,0.842857,0.819444,0.830986,0.530120,0.571429,0.550000,0.686489,0.695437,0.690493,0.760670,0.754266,0.757143
Average,0.750853,0.842946,0.813437,0.827852,0.524625,0.576299,0.548949,0.683785,0.694868,0.688401,0.759010,0.750853,0.754284


#### Error analysis

In [98]:
%autoreload
folder_path = f"../results/CheckThat"
mistral_results = pd.read_csv(f"{folder_path}/{HuggingFaceModel.MISTRAL_7B_INSTRUCT.name}/zeroshot1.csv", index_col=0)
mixtral_results = pd.read_csv(f"{folder_path}/{HuggingFaceModel.MIXTRAL_INSTRUCT.name}/zeroshot1.csv", index_col=0)
results = [mistral_results, mixtral_results]
models = [HuggingFaceModel.MISTRAL_7B_INSTRUCT, HuggingFaceModel.MIXTRAL_INSTRUCT]
generate_error_analysis_report(
    results=results,
    models=models,
    folder_path=folder_path,
    label_column_name="check_worthiness",
    text_column_name="tweet_text",
)

##################################################
#              MISTRAL_7B_INSTRUCT               #
#              False positives: 376              #
#              False negatives: 36               #
#              Empty predictions: 1              #
#             Wrong output format: 0             #
##################################################
#                MIXTRAL_INSTRUCT                #
#              False positives: 484              #
#              False negatives: 20               #
#              Empty predictions: 3              #
#            Wrong output format: 25             #
##################################################
#                     Total                      #
#              False positives: 568              #
#              False negatives: 51               #
#        Overlapping false positives: 292        #
#         Overlapping false negatives: 5         #
##################################################


## LORA finetuning

Using LORA to fine-tune the Mistral 7B Instruct model

### Claimbuster

#### Generate cross validation datasets

In [4]:
%autoreload
claimbuster = load_claimbuster_dataset("../data/ClaimBuster_Datasets/datasets")
datasets = generate_cross_validation_datasets(claimbuster, folder_path="../data/ClaimBuster_Datasets/crossval")


### Checkthat

#### Generate cross validation datasets

In [21]:
checkthat = load_check_that_dataset("../data/CheckThat2021Task1a")
datasets = generate_cross_validation_datasets(
    checkthat, 
    label_column="check_worthiness",
    folder_path="../data/CheckThat2021Task1a/crossval"
)